In [2]:
import pandas as pd
yield_dataset = pd.read_csv(r'C:\DATA ANALYSIS AND DATA SCIENCE PROJECTS\AgriGrow\data\Raw Data\crop_yield_data.csv')
yield_dataset.head()

,Crop,Crop_Year,Season,State,Area,Production,Annual_Rainfall,Fertilizer,Pesticide,Yield
0,Arecanut,1997,Whole Year,Assam,73814.0,56708,2051.4,7024878.38,22882.34,0.796087
1,Arhar/Tur,1997,Kharif,Assam,6637.0,4685,2051.4,631643.29,2057.47,0.710435
2,Castor seed,1997,Kharif,Assam,796.0,22,2051.4,75755.32,246.76,0.238333
3,Coconut,1997,Whole Year,Assam,19656.0,126905000,2051.4,1870661.52,6093.36,5238.051739
4,Cotton(lint),1997,Kharif,Assam,1739.0,794,2051.4,165500.63,539.09,0.420909


In [3]:
yield_dataset['Season'].unique()

array(['Whole Year ', 'Kharif     ', 'Rabi       ', 'Autumn     ',
       'Summer     ', 'Winter     '], dtype=object)

In [4]:
yield_dataset['State'].unique()

array(['Assam', 'Karnataka', 'Kerala', 'Meghalaya', 'West Bengal',
       'Puducherry', 'Goa', 'Andhra Pradesh', 'Tamil Nadu', 'Odisha',
       'Bihar', 'Gujarat', 'Madhya Pradesh', 'Maharashtra', 'Mizoram',
       'Punjab', 'Uttar Pradesh', 'Haryana', 'Himachal Pradesh',
       'Tripura', 'Nagaland', 'Chhattisgarh', 'Uttarakhand', 'Jharkhand',
       'Delhi', 'Manipur', 'Jammu and Kashmir', 'Telangana',
       'Arunachal Pradesh', 'Sikkim'], dtype=object)

In [5]:
yield_dataset['Crop'].unique()

array(['Arecanut', 'Arhar/Tur', 'Castor seed', 'Coconut ', 'Cotton(lint)',
       'Dry chillies', 'Gram', 'Jute', 'Linseed', 'Maize', 'Mesta',
       'Niger seed', 'Onion', 'Other  Rabi pulses', 'Potato',
       'Rapeseed &Mustard', 'Rice', 'Sesamum', 'Small millets',
       'Sugarcane', 'Sweet potato', 'Tapioca', 'Tobacco', 'Turmeric',
       'Wheat', 'Bajra', 'Black pepper', 'Cardamom', 'Coriander',
       'Garlic', 'Ginger', 'Groundnut', 'Horse-gram', 'Jowar', 'Ragi',
       'Cashewnut', 'Banana', 'Soyabean', 'Barley', 'Khesari', 'Masoor',
       'Moong(Green Gram)', 'Other Kharif pulses', 'Safflower',
       'Sannhamp', 'Sunflower', 'Urad', 'Peas & beans (Pulses)',
       'other oilseeds', 'Other Cereals', 'Cowpea(Lobia)',
       'Oilseeds total', 'Guar seed', 'Other Summer Pulses', 'Moth'],
      dtype=object)

In [6]:
yield_dataset['Crop_Year'].unique()

array([1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007,
       2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018,
       2019, 2020])

In [7]:
print("Missing values per column:")
print(yield_dataset.isna().sum())

duplicates = yield_dataset.duplicated().sum()
print(f"\nNumber of duplicate rows: {duplicates}")

Missing values per column:
Crop               0
Crop_Year          0
Season             0
State              0
Area               0
Production         0
Annual_Rainfall    0
Fertilizer         0
Pesticide          0
Yield              0
dtype: int64

Number of duplicate rows: 0


In [8]:
missing_crop_years = (
    yield_dataset.groupby('Crop_Year')['Crop'].nunique().reset_index(name='Unique_Crops')
)
print("\nCrops available per year:")
print(missing_crop_years)


Crops available per year:
    Crop_Year  Unique_Crops
0        1997            50
1        1998            52
2        1999            52
3        2000            53
4        2001            53
5        2002            53
6        2003            53
7        2004            53
8        2005            53
9        2006            54
10       2007            53
11       2008            54
12       2009            54
13       2010            54
14       2011            54
15       2012            54
16       2013            54
17       2014            54
18       2015            54
19       2016            55
20       2017            55
21       2018            55
22       2019            55
23       2020            29


In [9]:
yield_dataset = yield_dataset[yield_dataset['Crop_Year'] != 2020]

In [10]:
rainfall_state_df = pd.read_csv(r'C:\DATA ANALYSIS AND DATA SCIENCE PROJECTS\AgriGrow\data\clean data\final_statewise_seasonal_rainfall.csv')
temp_state_df = pd.read_csv(r'C:\DATA ANALYSIS AND DATA SCIENCE PROJECTS\AgriGrow\data\clean data\statewise_seasonal_temperature.csv')

In [11]:
yield_dataset.rename(columns={'Crop_Year': 'Year'}, inplace=True)


In [12]:
merged_df = pd.merge(yield_dataset, rainfall_state_df, on=['State', 'Year'], how='left')

merged_df = pd.merge(merged_df, temp_state_df, on=['State', 'Year'], how='left')


In [13]:
import numpy as np
def get_seasonal_values(row, prefix):
    season = row['Season'].strip()  
    if season in row.index:
        return row[f"{season}_{prefix}"]
    return np.nan

merged_df.rename(columns=lambda x: x.replace('_x', '_rainfall').replace('_y', '_temp'), inplace=True)

def select_rainfall(row):
    season = row['Season'].strip()
    col = season if season in ['Rabi', 'Kharif', 'Whole Year', 'Autumn', 'Summer', 'Winter'] else None
    return row.get(f"{col}_rainfall", np.nan)

def select_temp(row):
    season = row['Season'].strip()
    col = season if season in ['Rabi', 'Kharif', 'Whole Year', 'Autumn', 'Summer', 'Winter'] else None
    return row.get(f"{col}_temp", np.nan)

merged_df['Seasonal_Rainfall'] = merged_df.apply(select_rainfall, axis=1)
merged_df['Seasonal_Temp'] = merged_df.apply(select_temp, axis=1)
columns_to_drop = [col for col in merged_df.columns if col.endswith('_rainfall') or col.endswith('_temp')]
merged_df.drop(columns=columns_to_drop, inplace=True)
merged_df.head(3)

,Crop,Year,Season,State,Area,Production,Annual_Rainfall,Fertilizer,Pesticide,Yield,ANNUAL,Seasonal_Rainfall,Seasonal_Temp
0,Arecanut,1997,Whole Year,Assam,73814.0,56708,2051.4,7024878.38,22882.34,0.796087,2372.2,2372.2,22.949286
1,Arhar/Tur,1997,Kharif,Assam,6637.0,4685,2051.4,631643.29,2057.47,0.710435,2372.2,1763.4,26.730655
2,Castor seed,1997,Kharif,Assam,796.0,22,2051.4,75755.32,246.76,0.238333,2372.2,1763.4,26.730655


In [14]:
merged_df.drop(columns=['Annual_Rainfall', 'ANNUAL'], inplace=True, errors='ignore')
merged_df.rename(columns={
    'Area': 'Area (ha)',
    'Production': 'Production (tonnes)',
    'Yield': 'Yield (t/ha)',
    'Fertilizer': 'Fertilizer (kg/ha)',
    'Pesticide': 'Pesticide (kg/ha)',
    'Seasonal_Rainfall': 'Seasonal_Rainfall (mm)',
    'Seasonal_Temp': 'Seasonal_Temperature (°C)'
}, inplace=True)
merged_df.head(3)

,Crop,Year,Season,State,Area (ha),Production (tonnes),Fertilizer (kg/ha),Pesticide (kg/ha),Yield (t/ha),Seasonal_Rainfall (mm),Seasonal_Temperature (°C)
0,Arecanut,1997,Whole Year,Assam,73814.0,56708,7024878.38,22882.34,0.796087,2372.2,22.949286
1,Arhar/Tur,1997,Kharif,Assam,6637.0,4685,631643.29,2057.47,0.710435,1763.4,26.730655
2,Castor seed,1997,Kharif,Assam,796.0,22,75755.32,246.76,0.238333,1763.4,26.730655


In [16]:
soil_df = pd.read_csv(r'C:\DATA ANALYSIS AND DATA SCIENCE PROJECTS\AgriGrow\data\clean data\statewise_soil_final.csv')
final_dataset = pd.merge(merged_df, soil_df, on='State', how='left')

In [17]:
final_dataset.head(3)

,Crop,Year,Season,State,Area (ha),Production (tonnes),Fertilizer (kg/ha),Pesticide (kg/ha),Yield (t/ha),Seasonal_Rainfall (mm),...,EC_NonSaline_%,EC_Saline_%,Sulphur_Sufficient_%,Sulphur_Deficient_%,Iron_Sufficient_%,Iron_Deficient_%,Zinc_Sufficient_%,Zinc_Deficient_%,Boron_Sufficient_%,Boron_Deficient_%
0,Arecanut,1997,Whole Year,Assam,73814.0,56708,7024878.38,22882.34,0.796087,2372.2,...,99.96,0.04,99.95,0.05,100.0,0.0,93.66,6.34,0.0,100.0
1,Arhar/Tur,1997,Kharif,Assam,6637.0,4685,631643.29,2057.47,0.710435,1763.4,...,99.96,0.04,99.95,0.05,100.0,0.0,93.66,6.34,0.0,100.0
2,Castor seed,1997,Kharif,Assam,796.0,22,75755.32,246.76,0.238333,1763.4,...,99.96,0.04,99.95,0.05,100.0,0.0,93.66,6.34,0.0,100.0


In [20]:
final_dataset.columns

Index(['Crop', 'Year', 'Season', 'State', 'Area (ha)', 'Production (tonnes)',
       'Fertilizer (kg/ha)', 'Pesticide (kg/ha)', 'Yield (t/ha)',
       'Seasonal_Rainfall (mm)', 'Seasonal_Temperature (°C)',
       'Nitrogen_High_%', 'Nitrogen_Medium_%', 'Nitrogen_Low_%',
       'Phosphorus_High_%', 'Phosphorus_Medium_%', 'Phosphorus_Low_%',
       'Potassium_High_%', 'Potassium_Medium_%', 'Potassium_Low_%',
       'Organic_Carbon_High_%', 'Organic_Carbon_Medium_%',
       'Organic_Carbon_Low_%', 'pH_Alkaline_%', 'pH_Acidic_%', 'pH_Neutral_%',
       'EC_NonSaline_%', 'EC_Saline_%', 'Sulphur_Sufficient_%',
       'Sulphur_Deficient_%', 'Iron_Sufficient_%', 'Iron_Deficient_%',
       'Zinc_Sufficient_%', 'Zinc_Deficient_%', 'Boron_Sufficient_%',
       'Boron_Deficient_%'],
      dtype='object')

In [22]:
final_dataset = final_dataset[[
    'Year', 'Crop', 'State', 'Season',
    'Seasonal_Temperature (°C)', 'Seasonal_Rainfall (mm)',
    'Fertilizer (kg/ha)', 'Pesticide (kg/ha)',
    'Nitrogen_High_%', 'Nitrogen_Medium_%', 'Nitrogen_Low_%',
    'Phosphorus_High_%', 'Phosphorus_Medium_%', 'Phosphorus_Low_%',
    'Potassium_High_%', 'Potassium_Medium_%', 'Potassium_Low_%',
    'Organic_Carbon_High_%', 'Organic_Carbon_Medium_%', 'Organic_Carbon_Low_%',
    'pH_Alkaline_%', 'pH_Acidic_%', 'pH_Neutral_%',
    'EC_NonSaline_%', 'EC_Saline_%',
    'Sulphur_Sufficient_%', 'Sulphur_Deficient_%',
    'Iron_Sufficient_%', 'Iron_Deficient_%',
    'Zinc_Sufficient_%', 'Zinc_Deficient_%',
    'Boron_Sufficient_%', 'Boron_Deficient_%',
    'Area (ha)', 'Production (tonnes)', 'Yield (t/ha)'
]]
final_dataset.head()


,Year,Crop,State,Season,Seasonal_Temperature (°C),Seasonal_Rainfall (mm),Fertilizer (kg/ha),Pesticide (kg/ha),Nitrogen_High_%,Nitrogen_Medium_%,...,Sulphur_Deficient_%,Iron_Sufficient_%,Iron_Deficient_%,Zinc_Sufficient_%,Zinc_Deficient_%,Boron_Sufficient_%,Boron_Deficient_%,Area (ha),Production (tonnes),Yield (t/ha)
0,1997,Arecanut,Assam,Whole Year,22.949286,2372.2,7024878.38,22882.34,24.9,75.04,...,0.05,100.0,0.0,93.66,6.34,0.0,100.0,73814.0,56708,0.796087
1,1997,Arhar/Tur,Assam,Kharif,26.730655,1763.4,631643.29,2057.47,24.9,75.04,...,0.05,100.0,0.0,93.66,6.34,0.0,100.0,6637.0,4685,0.710435
2,1997,Castor seed,Assam,Kharif,26.730655,1763.4,75755.32,246.76,24.9,75.04,...,0.05,100.0,0.0,93.66,6.34,0.0,100.0,796.0,22,0.238333
3,1997,Coconut,Assam,Whole Year,22.949286,2372.2,1870661.52,6093.36,24.9,75.04,...,0.05,100.0,0.0,93.66,6.34,0.0,100.0,19656.0,126905000,5238.051739
4,1997,Cotton(lint),Assam,Kharif,26.730655,1763.4,165500.63,539.09,24.9,75.04,...,0.05,100.0,0.0,93.66,6.34,0.0,100.0,1739.0,794,0.420909


In [23]:
final_dataset.to_csv("final_dataset.csv", index=False)